In [1]:
!pip install git+https://github.com/huggingface/transformers # need to install from github

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-308xxqj2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-308xxqj2
  Resolved https://github.com/huggingface/transformers to commit 59cd9de39da4c406e12c1785f70ee73806ebc6ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8344499 sha256=54d33b54c4b9e98c60bf6b085336f16e7c35ed728109689d7777690ef2991086
  Stored in directory: /tmp/pip-ephem-wheel-cache-ngca9mbc/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install datasets loralib sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 25.2 MB/s eta 0:00:00


In [4]:
!pip  install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00


## HuggingFace

There are two Hugging Face LLM wrappers, one for a local pipeline and one for a model hosted on Hugging Face Hub. Note that these wrappers only work for models that support the following tasks: text2text-generation, text-generation


## Loading Alpaca7B

In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch


In [12]:
import requests

def translate_text(text, target_language='ar'):
    url = f'https://api.mymemory.translated.net/get?q={text}&langpair=en|{target_language}'
    response = requests.get(url)
    translation = response.json()['responseData']['translatedText']
    return translation


In [2]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", load_in_8bit=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=12000,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
from langchain import PromptTemplate, LLMChain

template = """You are an adept Islamic expert assistant,
proficient in providing subtle guidance for intricate questions,
 assisting students in navigating towards the intended answers without outright disclosure.
  Below is the question the student is tasked to answer along with their response.
  Your role is to craft a response that effectively guides the student to a more comprehensive and refined answer,
  maintaining the essence of critical thinking and analytical exploration.
### Question :
{qst}
### Student answer:
{answer}

Hint:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [10]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

In [13]:

question = """Examine the intricate principles of Islamic finance,
providing a nuanced analysis of their philosophical underpinnings and practical applications in the context
 of contemporary economic practices. Delve into the complexities of how these principles align
 with and diverge from conventional finance, elucidating key differences. Additionally,
 explore potential areas of adaptation or evolution required to harmonize Islamic finance with the dynamic
 and multifaceted landscape of modern economies."""
answer = "i have no idea"

res = llm_chain.run({"answer":answer,"qst":question})
arabic_res = translate_text(res)
arabic_res

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'يجب توجيه الطالب لدراسة تعقيدات التمويل الإسلامي من خلال دراسة مبادئه الأساسية، مثل حظر الفائدة (الربا)، ومفهوم تقاسم المخاطر (المشاركة) وأهمية الاستثمارات المتوافقة مع الشريعة الإسلامية. يمكنهم بعد ذلك الخوض في الطرق التي تختلف بها هذه المبادئ عن التمويل التقليدي، بالإضافة إلى التحديات التي تواجههم عند محاولة دمجهم في الاقتصادات الحديثة.'

In [ ]:
question = "What are alpacas? and how are they different from llamas?"

print(llm_chain.run(question))